# Ejercicio práctico — Extracción de Entidades en Noticias en Español

Este cuaderno implementa paso a paso el ejercicio que compara dos enfoques de **Named Entity Recognition (NER)**:
1. Un sistema *rule‑based* minimalista (regex + gazetteer).
2. Un modelo Transformer pre‑entrenado y ajustado para NER en español.

**Objetivos de aprendizaje**
- Comprender el concepto de *entidad nombrada* y su papel en PLN.
- Implementar y evaluar un detector de entidades basado en reglas.
- Aplicar un modelo de estado‑del‑arte y comparar resultados.
- Analizar errores y reflexionar sobre la idoneidad de cada enfoque.

## 1 · Instalación de dependencias
Ejecuta la siguiente celda para instalar las bibliotecas necesarias. *(En Colab se tarda < 1 min).*

In [4]:
!pip install -q datasets seqeval fugashi ipadic

## 2 · Carga del corpus CoNLL‑2002 (español)
Tomaremos una **muestra pequeña** para acelerar los experimentos; puedes aumentar el número de frases si tu entorno lo permite.

In [5]:
from datasets import load_dataset

conll_es = load_dataset('conll2002', 'es')
train_ds = conll_es['train'].select(range(2000))   # 2 000 oraciones
test_ds  = conll_es['test'].select(range(500))     #   500 oraciones

print(train_ds[0]['tokens'])
print(train_ds[0]['ner_tags'])

/Users/carlos.schiaffino/projects/cursos/seminario-algosups.github.io/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1518/1518 [00:00<00:00, 21158.01 examples/s]

['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
[5, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0]


## 3 · Enfoque A — Reglas + Gazetteer
### 3.1 Crear un **gazetteer** minimalista
Añade, modifica o amplía las listas para experimentar.

In [6]:
import re, random
from itertools import chain

GAZETTEER_LOC = {
    'Madrid', 'Buenos Aires', 'Barcelona', 'Ciudad de México',
    'Sevilla', 'Valencia', 'Montevideo', 'Santiago',
    'Bogotá', 'Lima', 'Caracas', 'Asunción', 'Quito',
}

# Regex muy simple para años de cuatro dígitos
DATE_RE = re.compile(r'\b\d{4}\b')

### 3.2 Función de NER basada en reglas
Completa `rule_based_ner(tokens)` de modo que devuelva una lista de etiquetas BIO alineada con `tokens`.

> **TIP**: Etiquetas válidas en CoNLL‑2002: `B-LOC`, `I-LOC`, `B-PER`, `I-PER`, `B-ORG`, `I-ORG`, `B-MISC`, `I-MISC`, `O`.

In [7]:
def rule_based_ner(tokens):
    labels = []
    for tok in tokens:
        if tok in GAZETTEER_LOC:
            labels.append('B-LOC')
        elif DATE_RE.match(tok):
            labels.append('B-MISC')  # usamos MISC para fechas
        else:
            labels.append('O')
    return labels

# Ejemplo rápido
example = ['Barcelona', 'ganó', 'en', '1992', '.']
print(rule_based_ner(example))

['B-LOC', 'O', 'O', 'B-MISC', 'O']


### 3.3 Evaluación del sistema de reglas

In [8]:
from seqeval.metrics import classification_report, f1_score

y_true, y_pred = [], []
for row in test_ds:
    y_true.append([test_ds.features['ner_tags'].feature.names[tag] for tag in row['ner_tags']])
    y_pred.append(rule_based_ner(row['tokens']))

print(classification_report(y_true, y_pred))
print('F1 macro:', f1_score(y_true, y_pred))

              precision    recall  f1-score   support

         LOC       0.45      0.07      0.12       303
        MISC       0.00      0.00      0.00       117
         ORG       0.00      0.00      0.00       480
         PER       0.00      0.00      0.00       231

   micro avg       0.27      0.02      0.03      1131
   macro avg       0.11      0.02      0.03      1131
weighted avg       0.12      0.02      0.03      1131

F1 macro: 0.0347682119205298


/Users/carlos.schiaffino/projects/cursos/seminario-algosups.github.io/.venv/lib/python3.13/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 4 · Enfoque B — Modelo Transformer pre‑entrenado
Utilizaremos el modelo `mrm8488/bert-spanish-cased-finetuned-ner` disponible en Hugging Face. Puedes cambiarlo por otro para comparar.

In [12]:
!pip install -q transformers torch

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_ckpt = 'mrm8488/bert-spanish-cased-finetuned-ner'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForTokenClassification.from_pretrained(model_ckpt)

# Creamos pipeline con agrupación de tokens en spans (simple)
ner_pipe = pipeline('token-classification', model=model, tokenizer=tokenizer, aggregation_strategy='simple')

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


### 4.1 Conversión de la salida del pipeline a BIO tags
El siguiente helper transforma los spans del pipeline a etiquetas BIO alineadas con los tokens originales.

In [14]:
def pipeline_to_bio(tokens, spans):
    labels = ['O'] * len(tokens)
    for sp in spans:
        ent_tokens = tokenizer.tokenize(sp['word'])  # tokenización aproximada
        # buscamos la primera ocurrencia
        try:
            start_idx = tokens.index(ent_tokens[0])
        except ValueError:
            continue
        labels[start_idx] = f"B-{sp['entity_group'].upper()}"
        for j in range(1, len(ent_tokens)):
            if start_idx + j < len(labels):
                labels[start_idx + j] = f"I-{sp['entity_group'].upper()}"
    return labels

### 4.2 Etiquetado del conjunto de prueba y evaluación

In [15]:
y_true, y_pred = [], []
for row in test_ds:
    tokens = row['tokens']
    spans = ner_pipe(' '.join(tokens))
    y_true.append([test_ds.features['ner_tags'].feature.names[tag] for tag in row['ner_tags']])
    y_pred.append(pipeline_to_bio(tokens, spans))

print(classification_report(y_true, y_pred))
print('F1 macro:', f1_score(y_true, y_pred))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


              precision    recall  f1-score   support

         LOC       0.81      0.50      0.62       303
        MISC       0.49      0.29      0.37       117
         ORG       0.76      0.41      0.53       480
         PER       0.45      0.21      0.29       231

   micro avg       0.69      0.38      0.49      1131
   macro avg       0.63      0.35      0.45      1131
weighted avg       0.68      0.38      0.49      1131

F1 macro: 0.49232518476407044


## 5 · Comparación y discusión
Completa la siguiente tabla (manualmente o con código) copiando las métricas obtenidas.

| Sistema | F1 macro | PER | ORG | LOC | MISC |
|---------|----------|-----|-----|-----|------|
| Reglas  |          |     |     |     |      |
| BERT    |          |     |     |     |      |

Reflexiona:
- ¿En qué tipos de entidad se nota mayor diferencia?
- ¿Qué errores comete el sistema de reglas?
- ¿Por qué el modelo neuronal podría fallar en ciertos nombres propios poco frecuentes?

## 6 · Análisis cualitativo de errores
Extrae ejemplos donde el modelo Transformer falló. Clasifica cada error como *boundary*, *mis‑class* o *miss*.

In [16]:
import random, pandas as pd

error_samples = []
for row, pred in zip(test_ds, y_pred):
    gold = [test_ds.features['ner_tags'].feature.names[tag] for tag in row['ner_tags']]
    if gold != pred:
        error_samples.append({'text': ' '.join(row['tokens']), 'gold': gold, 'pred': pred})

sampled = random.sample(error_samples, 20)
pd.DataFrame(sampled)

,text,gold,pred
0,"La Coruña , 23 may ( EFECOM ) .","[B-LOC, I-LOC, O, O, O, O, B-ORG, O, O]","[B-LOC, I-LOC, O, O, O, O, O, O, O]"
1,"Al acto acudieron , además de los accionistas ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"Fuera de ambas alianzas dentro de la CEI , eco...","[O, O, O, O, O, O, O, B-ORG, O, O, O, O, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"Madrid , 23 may ( EFE ) .","[B-LOC, O, O, O, O, B-ORG, O, O]","[B-LOC, O, O, O, O, O, O, O]"
4,"Por su parte , el secretario provincial del PS...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, ...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, ..."
5,Según la información facilitada hoy por la Jef...,"[O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6,Otro ausente será el defensa Ned Zelic ( 1860 ...,"[O, O, O, O, O, B-PER, I-PER, O, O, B-LOC, O, ...","[O, O, O, O, O, B-PER, I-PER, O, O, O, O, B-LO..."
7,"El presidente de Aceralia , José Ramón Alvarez...","[O, O, O, B-ORG, O, B-PER, I-PER, I-PER, I-PER...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
8,Una treintena de asociaciones que integran la ...,"[O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER...","[O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER..."
9,"Rusia , a su vez , rechaza la intención de sus...","[B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O,..."


In [ ]:
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy
import random




# Crea un nlp vacío solo para construir Doc/Span
nlp_blank = spacy.blank("es")

def bio_to_spacy_doc(tokens, labels):
    """Convierte tokens + BIO en un Doc con entidades SpaCy."""
    doc = Doc(nlp_blank.vocab, words=tokens)
    spans = []
    i = 0
    while i < len(labels):
        if labels[i].startswith("B-"):
            ent_type = labels[i][2:]
            start = i
            i += 1
            while i < len(labels) and labels[i].startswith("I-"):
                i += 1
            spans.append(Span(doc, start, i, label=ent_type))
        else:
            i += 1
    doc.ents = spans
    return doc

# ─── muestra aleatoria ────────────────────────────────────────────
idx = random.randint(0, len(test_ds) - 1)
tokens = test_ds[idx]["tokens"]
gold_labels = [test_ds.features["ner_tags"].feature.names[t] for t in test_ds[idx]["ner_tags"]]
pred_labels = y_pred[idx]

print("🔹 Oro (gold):")
displacy.render(bio_to_spacy_doc(tokens, gold_labels), style="ent", jupyter=True)

print("🔸 Predicción BERT:")
displacy.render(bio_to_spacy_doc(tokens, pred_labels), style="ent", jupyter=True)

🔹 Oro (gold):


ImportError: cannot import name 'display' from 'IPython.core.display' (/Users/carlos.schiaffino/projects/cursos/seminario-algosups.github.io/.venv/lib/python3.13/site-packages/IPython/core/display.py)

## 7 · Conclusiones y trabajo futuro
- Resume fortalezas y limitaciones de cada método.
- Sugiere cómo mejoraría un **sistema híbrido** (p. ej. reglas específicas para fechas + modelo para el resto).
- ¿Qué pasos seguirías para adaptar el modelo a un dominio clínico?


---
### Créditos
- CoNLL‑2002 Shared Task: <https://www.clips.uantwerpen.be/conll2002/ner/>
- Modelo BERT‑NER en español: <https://huggingface.co/mrm8488/bert-spanish-cased-finetuned-ner>

© 2025 – Elaborado para la clase de **Algoritmos Supervisados y Anotación para PLN**.